# Cuboid point cloud visualization in Polyscope

In [1]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Define the dimensions of the box

In [6]:
length = 0.3 # [m]
width = 0.5 # [m]
height = 0.7 # [m]
resolution = 0.01 # [m]

In [12]:
def make_cuboid_pcd(l: float, w: float, h: float, res: float) -> o3d.geometry.PointCloud:
    # Create arrays for each dimension with the specified dimensions sampled at the specified resolution.
    x = np.arange(start=0., stop=l, step=res, dtype=np.float64) # array of length d_x
    y = np.arange(start=0., stop=w, step=res, dtype=np.float64) # array of length d_y
    z = np.arange(start=0., stop=h, step=res, dtype=np.float64) # array of length d_z
    # Face 1
    # We create a rectangular grid of x and y coordinates for the given length and width.
    xy_x, xy_y = np.meshgrid(x, y) # Both xy_x and xy_y have shape d_x by d_y
    xy_z_1 = np.zeros_like(xy_x)
    face_1_arr = np.vstack((np.reshape(xy_x, -1),
                        np.reshape(xy_y, -1),
                        np.reshape(xy_z_1, -1),
                        ))
    # Create a 3D point cloud using Open3D
    face_1_pcd = o3d.geometry.PointCloud()
    face_1_pcd.points = o3d.utility.Vector3dVector(face_1_arr.T)
    # Get the corresponding z-coordinates for the points in face 3
    xy_z_3 = h*np.ones_like(xy_x)
    face_3_arr = np.vstack((np.reshape(xy_x, -1),
                        np.reshape(xy_y, -1),
                        np.reshape(xy_z_3, -1),
                        ))
    # Create a 3D point cloud using Open3D
    face_3_pcd = o3d.geometry.PointCloud()
    face_3_pcd.points = o3d.utility.Vector3dVector(face_3_arr.T)
    yz_y, yz_z = np.meshgrid(y, z) # Both yz_y and yz_z have shape d_y by d_z

    # Face 2, which is aligned with the x=0 plane.
    yz_x_2 = np.zeros_like(yz_y)
    face_2_arr = np.vstack((np.reshape(yz_x_2, -1),
                        np.reshape(yz_y, -1),
                        np.reshape(yz_z, -1),
                        ))
    # Create a 3D point cloud using Open3D
    face_2_pcd = o3d.geometry.PointCloud()
    face_2_pcd.points = o3d.utility.Vector3dVector(face_2_arr.T)

    # Face 4.
    yz_x_4 = l*np.ones_like(yz_y)
    face_4_arr = np.vstack((np.reshape(yz_x_4, -1),
                        np.reshape(yz_y, -1),
                        np.reshape(yz_z, -1),
                        ))
    # Create a 3D point cloud using Open3D
    face_4_pcd = o3d.geometry.PointCloud()
    face_4_pcd.points = o3d.utility.Vector3dVector(face_4_arr.T)

    # Face 5
    xz_x, xz_z = np.meshgrid(x, z) # Both yz_x and yz_z have shape d_x by d_z
    # We create the corresponding grid of x coordinates for face 5, which is aligned with the y=0 plane.
    xz_y_5 = np.zeros_like(xz_x)
    face_5_arr = np.vstack((np.reshape(xz_x, -1),
                        np.reshape(xz_y_5, -1),
                        np.reshape(xz_z, -1),
                        ))
    # Create a 3D point cloud using Open3D
    face_5_pcd = o3d.geometry.PointCloud()
    face_5_pcd.points = o3d.utility.Vector3dVector(face_5_arr.T)

    # Face 6.
    xz_y_6 = w*np.ones_like(xz_x)
    face_6_arr = np.vstack((np.reshape(xz_x, -1),
                        np.reshape(xz_y_6, -1),
                        np.reshape(xz_z, -1),
                        ))
    # Create a 3D point cloud using Open3D
    face_6_pcd = o3d.geometry.PointCloud()
    face_6_pcd.points = o3d.utility.Vector3dVector(face_6_arr.T)
    cuboid_pcd = face_1_pcd + face_2_pcd + face_3_pcd + face_4_pcd + face_5_pcd + face_6_pcd
    # Remove duplicated points beloging to shared edges.
    cuboid_pcd.remove_duplicated_points()
    return cuboid_pcd


In [13]:
box_pcd = make_cuboid_pcd(length, width, height, resolution)
# Calculate normals to improve illumination in data visualization
box_pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=0.05,
            max_nn=16
        ),
        fast_normal_computation=True,
)
o3d.visualization.draw_geometries(
        geometry_list=[box_pcd],
        window_name='Cuboid Point Cloud',
        point_show_normal=False,)

In [14]:
import polyscope as ps

ps_radius=0.01

ps.init()
ps.set_program_name("Point cloud viz")
ps.set_up_dir("z_up")
ps_box = ps.register_point_cloud("Box PCD", np.asarray(box_pcd.points), enabled=True,
                                              material='candy',
                        radius=ps_radius, color=(1., 0., 0.),
                        transparency=0.5)
ps.show()